# Optical Communications Project -- Team 8

In [1]:
# fibre specification
n1 = 1.46
n2 = 1.45
lambda_ = 820e-9
a = 5.1e-6

In [2]:
#import relevant packages
import numpy as np
import matplotlib.pyplot as plt

## 1. Calculate the V-parameter for the fibre.

In [3]:
V = 2*np.pi*a/lambda_*np.sqrt(n1*n1-n2*n2)
print(V)

6.666266729164944


## 2. Decide whether you are going to consider the {TE, TM, EH, HE} or LP modes for all of the subsequent tasks.


## 3. Identify all the modes that the fibre supports and the propagation constant and effective index for each mode. Summarise the results as a table.

In [ ]:

def j(pa, m):
    u=-ss.jv(m+1, pa)+m*ss.jv(m, pa)/pa
    l=pa*ss.jv(m, pa)
    return u/l   #first order bessel function terms 

def k(qa, m):
    u=-ss.kn(m+1, qa)+m*ss.kn(m, qa)/qa 
    l= qa*ss.kn(m, qa)
    return u/l #Second order bessel terms

def b(pa): #Convert between pa values and Wavenumber 
    p=pa/a
    b=np.sqrt((n1*k0)**2-p**2)
    return b

def pa(b): #Convert between Wavenumber and pa values
    p=np.sqrt((n1*k0)**2-b**2)
    return p*a

def lhs(b, m): # LHS of quadratic solution
    pa=np.sqrt((n1*k0)**2-b**2)*a
    qa=np.sqrt(b**2-(n2*k0)**2)*a
    t1=(j(pa, m)+k(qa, m))
    t2=(j(pa, m)+n2**2*k(qa, m)/n1**2)

    return t1*t2

def rhs(b, m):# RHS of quadratic solution
    pa=np.sqrt((n1*k0)**2-b**2)*a
    qa=np.sqrt(b**2-(n2*k0)**2)*a
    t1=(1/(pa)**2+1/(qa)**2)
    t2=(1/(pa)**2+n2**2/(n1*qa)**2)
    return m**2*t1*t2 

def f(b):#Optimisation function
   return lhs(b, m1)-rhs(b, m1)

global m1

LHS=[]
RHS=[]
btest=np.linspace(n2*k0,n1*k0, 10000) #Potential b-values within constraints
mi=[0, 1, 2, 3, 4]
solfull=[]
sys.stderr = DevNull()
for m in mi:
    m1=m
    LHS=[]
    RHS=[]
    paVals=[]
    for b in btest: #Iterate through potential b values
        
        LHS.append(lhs(b, m))
        RHS.append(rhs(b, m))
    sol=sp.optimize.root(f, btest[20])
    sol3=sp.optimize.root(f, btest[int(len(btest)/2)])
    sol2=sp.optimize.root(f, btest[-30])
    solutions=list(set([sol.x[0], sol2.x[0], sol3.x[0]]))
    for s in solutions:
        plt.scatter(s,lhs(s, m), label="Intersections", s=20 )
        paVals.append(pa(s))
    
    solfull.append([solutions, paVals])
    plt.plot(btest, RHS, 'k', label="RHS")
    plt.plot(btest, LHS, 'b', label="LHS")
    plt.ylim(0, 10)
    plt.legend()
    plt.title("Mode order: "+str(m))
    plt.show()
    
print(solfull )

## 4. Excluding the fundamental mode {HE11/LP01}, pick one of the modes from your table for the tasks below.


## 5. Plot maps in the plane perpendicular to the fibre axis of the amplitude of all three E-fields within the fibre. You will need to fix one of the A or B parameters, e.g. to unity, and use the relevant equations to calculate the other values. You can choose to do this for either the {radial, tangential, z} or {x, y, z} E-field components.


## 6. Plot the spatial distribution of the total intensity of the mode in the direction perpendicular to the fibre axis, e.g. sum of the square modulus of the field in the {radial, tangential} or {x, y} directions.

## 7. Calculate numerically the waveguide dispersion $ 𝐷_w = − \frac{\lambda}{c} [\frac{\partial^2n_{eff}}{\partial\lambda^2}]_w = − \frac{\lambda}{c} [\frac{\partial^2n_{m}}{\partial\lambda^2}]_w $ for the mode selected, see notes for lecture 6. For example, calculate this from the effective index calculated for three (or more) wavelengths close to the wavelength of light to be transmitted.

## 8. Using your result to part 6, calculate the fraction of energy in the core and cladding by summing over the relevant regions. Use equation $𝑛^2_{eff} \approx 𝑛^2_1\Gamma(𝑉) + 𝑛^2_2(1 − \Gamma(𝑉)) $ to calculate an approximate value for the effective index. Compare this value of the effective index to the value you calculated directly and comment on your result.